## Data Augmentation

In [1]:
from scipy.io import wavfile as wav
from scipy.io.wavfile import read, write
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import librosa.display
from pathlib import Path
from sklearn.model_selection import train_test_split

In [2]:
def add_noise(file_path):
    data = librosa.core.load(file_path)[0]
    noise = np.random.randn(len(data))
    data_noise = data + 0.005 * noise
    return data_noise
 
def shift(file_path):
    data = librosa.core.load(file_path)[0]
    return np.roll(data, 300)
 
def stretch(file_path):
    data = librosa.core.load(file_path)[0]
    rate = 1.5
    data = librosa.effects.time_stretch(data, rate)
    return data
 
def write_audio_file(file, data, sample_rate=22050):
    librosa.output.write_wav(file, data, sample_rate)
    
    
def new_name (source_str, insert_str):
    #return source_str[:-4]+insert_str+source_str[-4:]
    return source_str[:34] + str('Train_augment_pad/') + source_str[34:-4] + insert_str + source_str[-4:]

def get_label(path):
    if path.parts[-3] == 'Shuffle':
        return 1
    else:
        return 0

def pad_signal(path, length):
    samples, sample_rate = librosa.load(path)
    name = path  # To rewrite files
    if len(samples) < length:
        y = np.pad(samples, (0, length-len(samples)), 'constant')
    else:
        y = samples
    return librosa.output.write_wav(path=name, y=y, sr=sample_rate)

def pad_signal_rename (path, length, new_folder):
    samples, sample_rate = librosa.load(path)
    name = str(path)[:34] + new_folder + str(path)[34:] # To rename files
    if len(samples) < length:
        y = np.pad(samples, (0, length-len(samples)), 'constant')
    else:
        y = samples
    return librosa.output.write_wav(path=name, y=y, sr=sample_rate)


### Getting all the cleaned data in one list

In [3]:
shuffle_col=[]
pathlist = Path('../../../Source/Clean_train_clips/Shuffle').glob('**/*.wav')
for path in pathlist:
    shuffle_col.append(path)

In [4]:
bc_col = []
pathlist = Path('../../../Source/Clean_train_clips/Ball_change').glob('**/*.wav')
for path in pathlist:
    bc_col.append(path)
    bc_col.sort()

In [5]:
path_col = shuffle_col + bc_col

### Set aside some data for validation

In [6]:
len(path_col)

343

In [ ]:
path_copy = path_col.copy()

In [ ]:
path_df = pd.DataFrame(path_copy, columns=['Path'])
path_df['Labels'] = [get_label(path_df.loc[idx,'Path']) for idx in range(len(path_df))]

In [ ]:
path_df.head()

In [ ]:
X = path_df['Path']
y = path_df['Labels']

In [ ]:
X_train, X_extra, y_train, y_extra = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.33,
                                                     random_state=17)

In [ ]:
X_train.shape, X_extra.shape
y_train.shape, y_extra.shape

In [ ]:
X_test, X_validate, y_test, y_validate = train_test_split(X_extra, y_extra,
                                                    stratify=y_extra, 
                                                    test_size=0.33,
                                                     random_state=17)

In [ ]:
X_train.shape, X_test.shape, X_validate.shape
y_train.shape, y_test.shape, y_validate.shape

In [ ]:
# Test Data for later use in padding
X_test.to_csv('../../../Source/Reserved_data/X_test_reserved.csv', index=None, header=True)
y_test.to_csv('../../../Source/Reserved_data/y_test_reserved.csv', index=None, header=True)

# Validate Data for later use in padding
X_validate.to_csv('../../../Source/Reserved_data/X_validate_reserved.csv', index=None, header=True)
y_validate.to_csv('../../../Source/Reserved_data/y_validate_reserved.csv', index=None, header=True)

In [ ]:
path_col = (X_train.tolist())
len(path_col)


### Creating, exporting noisy clips

In [ ]:
for i in range (len(path_col)): 
    write_audio_file((new_name(str(path_col[i]), '_noise')),
                      add_noise(path_col[i]))

### Creating, exporting stretched clips

In [ ]:
for i in range (len(path_col)): 
    write_audio_file((new_name(str(path_col[i]), '_stretch')),
                      stretch(path_col[i]))

### Creating, exporting shifted clips

In [ ]:
for i in range (len(path_col)): 
    write_audio_file((new_name(str(path_col[i]), '_shift')),
                      shift(path_col[i]))

### Export un-augmented data

In [ ]:
for i in range (len(path_col)):
    write_audio_file((new_name(str(path_col[i]), '_')),
                     librosa.core.load(path_col[i])[0])

### Pad samples with silence (to create equal lengths) and resave (overwrite existing files)

In [ ]:
path_col = []
pathlist = Path('../../../Source/Clean_train_clips/Train_augment_pad').glob('**/*.wav')
for path in pathlist:
    path_col.append(path)

In [ ]:
len(path_col) # should be 229*4=916

In [8]:
length_list = []
for i in range (len(path_col)):
    samples, sample_rate = librosa.load(path_col[i])
    length_list.append(len(samples))
max_length = max(length_list)
print(max_length)

20772


In [ ]:
for i in range(len(path_col)):
    pad_signal(path_col[i], max_length)

In [ ]:
# Check to see if it worked
for i in range (len(path_col)):
    samples, sample_rate = librosa.load(path_col[i])
    print(len(samples))

### Repeat padding for test and validate sets (and write files)

In [ ]:
# Test Data 
X_test = pd.read_csv('../../../Source/Reserved_data/X_test_reserved.csv')

# Validate Data 
X_validate = pd.read_csv('../../../Source/Reserved_data/X_validate_reserved.csv')

In [ ]:
val_col, test_col = [], []
for i in range(len(X_validate)):
    val_col.append(Path(X_validate.loc[i, 'Path']))
    
for j in range(len(X_test)):
    test_col.append(Path(X_test.loc[j, 'Path']))

In [ ]:
print (len(val_col) == len(X_validate))
print (len(test_col) == len(X_test))
print (max_length)

In [ ]:
for i in range(len(val_col)):
    pad_signal_rename(val_col[i], max_length, 'Validate_pad/')

In [ ]:
for i in range(len(test_col)):
    pad_signal_rename(test_col[i], max_length, 'Test_pad/')

In [ ]:
# Check to see if it worked
path_col = []
pathlist = Path('../../../Source/Clean_train_clips/Test_pad').glob('**/*.wav') # For Test_pad and Validate_pad
for path in pathlist:
    path_col.append(path)

for i in range (len(path_col)):
    samples, sample_rate = librosa.load(path_col[i])
    print(len(samples))

In [9]:
for i in range(len(path_col)):
    pad_signal_rename(path_col[i], max_length, 'All_data_pad/')

FileNotFoundError: [Errno 2] No such file or directory: '../../../Source/Clean_train_clips/All_data_pad/Shuffle/7/16.wav'